In [ ]:
!nvidia-smi

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [9]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization, Conv2D, Conv2DTranspose, LeakyReLU, Activation, Flatten, Dense, Input, MaxPool2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, SeparableConv2D, Add, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import Sequential, losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import load_img, img_to_array
from matplotlib import pyplot as plt

# from google.colab.patches import cv2_imshow

import numpy as np
import os
import cv2
import pandas as pd
import json
import tensorflow as tf
import math
import time

In [ ]:
!pip install opencv-python

In [10]:
from tensorflow.keras.layers import Layer

class SSIMLayer(Layer):

  def __init__(self):
    super(SSIMLayer, self).__init__()

  def call(self, inputs):
    img1, img2 = inputs
    ssim = tf.image.ssim_multiscale(img1, img2, 1.0)
    return ssim

class AutoencoderBPR4(Model):
  def __init__(self):
    super(AutoencoderBPR4, self).__init__()
    self.encoder = Sequential([
      Input(shape=(224, 224, 1)),
      Conv2D(32, (3, 3), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((2,2)),
      Conv2D(32, (4, 4), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((2,2)),
      Conv2D(32, (3, 3), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((1,1)),
      Conv2D(32*2, (4, 4), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((2,2)),
      Conv2D(32*2, (3, 3), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((1,1)),
      Conv2D(32*4, (4, 4), activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((2,2)),
      Conv2D(32*2, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((1,1)),
      Conv2D(32, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      MaxPooling2D((1,1)),
      Conv2D(256, (8, 8), strides=1, activation='linear', padding='valid'),
      ])

    self.decoder = Sequential([
      Conv2DTranspose(32, (8, 8), strides=1, activation=LeakyReLU(alpha=0.2), padding='valid'),
      Conv2D(32*2, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2D(32*4, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2DTranspose(32*2, (4, 4), strides=2, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2D(32*2, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2DTranspose(32, (4, 4), strides=2, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2D(32, (3, 3), strides=1, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2DTranspose(32, (4, 4), strides=2, activation=LeakyReLU(alpha=0.2), padding='same'),
      Conv2DTranspose(1, (4, 4), strides=2, activation='sigmoid', padding='same'),
      ])
    
    self.ssim_layer = SSIMLayer()
  
    self.optimizer = Adam(learning_rate=1e-04)

  
  def call(self, x):
    anchor, img_test = x

    encoded_pos = self.encoder(anchor)
    decoded_pos = self.decoder(encoded_pos)
    ssim_pos = self.ssim_layer([anchor, decoded_pos])

    encoded_neg = self.encoder(img_test)
    decoded_neg = self.decoder(encoded_neg)
    ssim_neg = self.ssim_layer([anchor, decoded_neg])

    return ssim_pos, ssim_neg


    
#DEVI CALCOLARE SSIM TRA IMMAGINE CHE HO DATO COMWE NEG E LA POS
  
  @tf.function
  def train_step(self, data):
    img, neg = data

    with tf.GradientTape() as tape:
      ssim_pos, ssim_neg = self(data, training=True)
      
      difference = tf.clip_by_value(ssim_pos - ssim_neg, -1, 1)

      loss = tf.reduce_sum(tf.nn.softplus(-difference))

    # Compute gradients
    trainable_vars = self.trainable_variables
    gradients = tape.gradient(loss, trainable_vars)

    # Update weights
    self.optimizer.apply_gradients(zip(gradients, trainable_vars))

    return {"loss": loss}
    

  @tf.function
  def test_step(self, data):
    # Unpack the data
    img, neg = data
    
    ssim_pos, ssim_neg = self(data, training=False)
    
    difference = tf.clip_by_value(ssim_pos - ssim_neg, -1, 1)
    test_loss = tf.reduce_sum(tf.nn.softplus(-difference))

    return {"test loss": test_loss}


  @tf.function
  def predict(self, data):
    ssim1, ssim2 = self(data, training=False)
    return ssim1, ssim2

autoencoderBPR4 = AutoencoderBPR4()
autoencoderBPR4.build([[None, 224, 224, 1], [None, 224, 224, 1]])
# autoencoderBPR4.load_weights('/content/autoencoder3108_1.hdf5')
autoencoderBPR4.load_weights('/content/autoencoderBPR4_19_09.hdf5')

In [11]:
def load_img_mine(img_dir, img_list):
  images = []
  for i, img_name in enumerate(img_list):
    if img_name.split('.')[1] == 'png':
      t1 = time.time()
      image = (load_img(os.path.join(img_dir, img_name), color_mode='grayscale'))
      t2 = time.time()
      image = img_to_array(image)
      t3 = time.time()
      image = np.divide(image, 255)
      t4 = time.time()
      image = image.astype('float32')
      t5 = time.time()
      #image = (img_to_array(load_img(os.path.join(img_dir, img_name), color_mode='grayscale'))/255).astype('float32')
      images.append(image)
      t6 = time.time()
      # t1 = time.time()
      # image = cv2.imread(os.path.join(img_dir, img_name), 0)
      # t2 = time.time()
      # image = np.divide(image, 255).astype('float32')
      # t3 = time.time()
      # images.append(image)
      # t4 = time.time()

      # print(f"load image in: {t2-t1}, divided: {t3-t2}, append: {t4-t3}")
#       if i < 10:
#         print(f"load image in: {t2-t1}, img_to_array: {t3-t2}, divided: {t4-t3}, astype: {t5-t4}, append: {t6-t5}")

  return np.array(images)



def dataLoaderImagesAutoencoderBPR(img_dir, img_list, gt_dir, gt_list, batch_size, random_seed=42):
  assert len(img_list) == len(gt_list)
  L = len(img_list)

  while True:
    batch_start = 0
    batch_end = batch_size

    while batch_start < L:
      limit = min(batch_end, L)

      if not len(img_list[batch_start:limit]) == batch_size:
        batch_start += batch_size
        batch_end += batch_size
        continue

      x_img = load_img_mine(img_dir, img_list[batch_start:limit])
      y_img = load_img_mine(gt_dir, gt_list[batch_start:limit])

      yield (x_img, y_img)

      batch_start += batch_size
      batch_end += batch_size

In [ ]:
i = cv2.imread('new_dataset_9000_11_08/new_colored/colored_0001_0.png')
# cv2.imshow("ssss",i)
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import cv2
from matplotlib import pyplot as plt

plt.imshow(i)
plt.show()

In [12]:
%matplotlib inline

def run_inference_time(img_dir, index, autoencoder_model, start_index, stop_index):
  
    max_tot, index_tot, count = 0, 0, 0

    list_item = sorted(os.listdir(img_dir))[index]
    im_test = (img_to_array(load_img(os.path.join(img_dir, list_item), color_mode='grayscale'))/255).astype('float32')


    loop_list = sorted(os.listdir(img_dir))[start_index:stop_index]
    initial = len(loop_list)

    if list_item in loop_list:
        loop_list.remove(list_item) 

    print(f"list_item {list_item}, loop_list_length {initial} --> {len(loop_list)}")

    batch_size = 128

    dataloader_train_bpr = dataLoaderImagesAutoencoderBPR(img_dir, loop_list, img_dir, loop_list, batch_size)

    steps_per_epoch = len(loop_list) // batch_size 

    for i in range(steps_per_epoch):
        t1 = time.time()
        test_im, _ = dataloader_train_bpr.__next__()
        t2 = time.time()
        count += batch_size

        _, ssimi = autoencoder_model.predict([np.expand_dims(im_test, 0), test_im])
        # print(ssimi)
        t3 = time.time()

        for j, s in enumerate(ssimi):
            if max_tot < s:
                max_tot = s
                # index_tot = i
                final_im = test_im[j]

        print(f"time required for reading: {t2 - t1}, time required for inference: {t3 - t2}")
        print("_______________________________________________________________________________________________________________")

    # final_im = (img_to_array(load_img(os.path.join(img_dir, final_im), color_mode='grayscale'))/255).astype('float32')

    #   plt.imshow((im_test*255).astype('uint8'))
    #   plt.show()

    #   plt.imshow((final_im*255).astype('uint8'))
    #   plt.show()

    print(max_tot)
    print(count)


    return im_test, final_im, max_tot


In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
t1 = time.time()
im_test, final_im, max_ssim = run_inference_time('/content/new_dataset_9000_11_08/new_colored', 1000, autoencoderBPR4)
t2 = time.time()
print(t2-t1)

In [23]:
N = 4
elements = len(sorted(os.listdir('/content/new_dataset_9000_11_08/new_colored')))
index_bound = math.floor(elements/N)

index_list = []

for i in range(N):
    start = i*index_bound
    stop = start + index_bound
    
    if i == (N-1):
        stop = elements

    index_list.append((start, stop))

print(index_list)

[(0, 7302), (7302, 14604), (14604, 21906), (21906, 29209)]


In [ ]:
import threading

class myThread (threading.Thread):
    def __init__(self, indexes, name):
        threading.Thread.__init__(self)
        self.indexes = indexes
        self.name = name

    def run(self):
        print(f"Starting {self.name}")
        t1 = time.time()
        im_test, final_im, max_ssim = run_inference_time('/content/new_dataset_9000_11_08/new_colored', 10000, autoencoderBPR4, self.indexes[0], self.indexes[1])
        print(f"i am in thread and this is max ssim {max_ssim}")
        self.im_test, self.final_im, self.max_ssim = im_test, final_im, max_ssim
        t2 = time.time()
        print(f"Exiting {self.name}, time spent: {t2-t1}")


thread_list = []
for t in range(N):
    thread_list.append(myThread(index_list[t], "thread_{}".format(t+1)))

for t in thread_list:
    t.start()
    
for t in thread_list:    
    t.join()
            
max_ssim = 0
max_thread = None
for t in thread_list:
    if t.max_ssim > max_ssim:
        max_ssim = t.max_ssim
        max_thread = t

in_im, fin_im = t.im_test, t.final_im
plt.imshow((in_im*255).astype('uint8'))
plt.show()
plt.imshow((fin_im*255).astype('uint8'))
plt.show()

Starting thread_1
Starting thread_2
Starting thread_3
Starting thread_4
list_item colored_3342_2.png, loop_list_length 7302 --> 7302
list_item colored_3342_2.png, loop_list_length 7302 --> 7301
list_item colored_3342_2.png, loop_list_length 7302 --> 7302
list_item colored_3342_2.png, loop_list_length 7303 --> 7303
time required for reading: 0.6837611198425293, time required for inference: 0.47168469429016113
_______________________________________________________________________________________________________________
time required for reading: 0.6767621040344238, time required for inference: 0.5504796504974365
_______________________________________________________________________________________________________________
time required for reading: 0.643573522567749, time required for inference: 0.49993228912353516
_______________________________________________________________________________________________________________
time required for reading: 0.7169146537780762, time required f